In [14]:
import os
import sys

In [17]:
%pwd

'g:\\My Drive\\Text Summarizer'

In [18]:
os.chdir("../")

In [20]:
sys.path.append(os.path.abspath('G:/My Drive/Text Summarizer/src'))

In [21]:
from textsummarizer.constants import *

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from textsummarizer.utils.common import read_yaml, create_directories

In [25]:
class Configurationmanager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(

                root_dir=config.root_dir,
                source_url=config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config

In [ ]:
import urllib.request as request
import zipfile
from textsummarizer.loggin import logger
from textsummarizer.utils.common import get_size